In [1]:
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from strats.btc_spec import BTCSpec
import numpy as np
import pickle

In [2]:
start_year = datetime.now().year
end_year = start_year + 1
training_year = 4

In [3]:
strat_class = BTCSpec(training_year)
strat_class.pull_included_columns()
strat_class.pull_factors()

In [4]:
market = Market()

In [5]:
market.connect()
ticker = "BTC"
data = []
prices = market.retrieve("pdr_crypto")
prices = p.column_date_processing(prices)
try:
    prices["year"] = [x.year for x in prices["date"]]
    prices["week"] = [x.week for x in prices["date"]]
    ticker_data = prices.copy()
    ticker_data.sort_values("date",ascending=True)
    complete = strat_class.transform_live(ticker_data,ticker)
    data.append(complete[complete["year"]==start_year])
except:
    print("whut")
market.disconnect()
final_data = pd.concat(data)

In [6]:
final_data = final_data.groupby(["date","ticker"]).mean().reset_index()

In [7]:
final_data

,date,ticker,high,low,open,close,volume,adjclose,year,quarter,...,7,8,9,10,11,12,13,d1,d2,d3
0,2022-01-01,BTC,47827.312500,46288.484375,46311.746094,47686.812500,2.458267e+10,47686.812500,2022.0,1.0,...,50429.859375,50822.195312,50784.539062,48628.511719,48936.613281,46880.277344,46707.015625,0.029809,-2.613381,0.204297
1,2022-01-02,BTC,47881.406250,46856.937500,47680.925781,47345.218750,2.795157e+10,47345.218750,2022.0,1.0,...,50809.515625,50429.859375,50822.195312,50784.539062,48628.511719,48936.613281,46880.277344,-0.007163,-1.240303,-0.525403
2,2022-01-03,BTC,47510.726562,45835.964844,47343.542969,46458.117188,3.307163e+10,46458.117188,2022.0,1.0,...,50640.417969,50809.515625,50429.859375,50822.195312,50784.539062,48628.511719,48936.613281,-0.018737,1.615686,-2.302655
3,2022-01-04,BTC,47406.546875,45752.464844,46458.851562,45897.574219,4.249468e+10,45897.574219,2022.0,1.0,...,47588.855469,50640.417969,50809.515625,50429.859375,50822.195312,50784.539062,48628.511719,-0.012066,-0.356053,-1.220373
4,2022-01-05,BTC,46929.046875,42798.222656,45899.359375,43569.003906,3.685108e+10,43569.003906,2022.0,1.0,...,46444.710938,47588.855469,50640.417969,50809.515625,50429.859375,50822.195312,50784.539062,-0.050734,3.204868,-10.001095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,2022-08-27,BTC,20336.714844,20051.068359,20249.454102,20180.695312,3.648767e+10,20180.695312,2022.0,3.0,...,21456.437500,21522.971680,21528.087891,21538.428711,21367.958984,21293.028320,21398.908203,-0.005367,-3.347314,1.688795
239,2022-08-28,BTC,20139.054688,19616.814453,20041.035156,19616.814453,2.436681e+10,19616.814453,2022.0,3.0,...,21497.961914,21395.019531,21456.437500,21522.971680,21528.087891,21538.428711,21367.958984,-0.010601,-0.224911,-1.957514
240,2022-08-29,BTC,20357.462891,19600.785156,19615.154297,20297.994141,3.263785e+10,20297.994141,2022.0,3.0,...,20923.488281,21593.930664,21600.904297,21497.961914,21395.019531,21456.437500,21522.971680,0.001742,-0.944050,-0.437000
241,2022-08-30,BTC,20542.644531,19617.640625,20298.611328,19796.808594,3.448336e+10,19796.808594,2022.0,3.0,...,20041.738281,20319.652344,20260.019531,21586.957031,21600.904297,21600.904297,21395.019531,0.000000,-1.000000,-1.000000


In [8]:
strat_class.db.connect()
models = strat_class.db.retrieve("models")
models["model"] = [pickle.loads(x) for x in models["model"]]
for year in tqdm(range(start_year,start_year+1)):
    try:
        ticker_data = final_data.copy()
        prediction_set = ticker_data[ticker_data["year"]==year].dropna()
        prediction_set = m.predict(models,prediction_set,strat_class.factors)
        prediction_set = strat_class.prediction_clean(prediction_set)
        strat_class.db.store("recs",prediction_set.iloc[-2:-1])
    except Exception as e:
        print(year,str(e))
strat_class.db.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 27.77it/s]
